<a href="https://colab.research.google.com/github/ipeirotis/sql_autograding/blob/main/few_shot_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai
!pip install -q google-cloud-secret-manager
!pip3 install -U -q PyMySQL sqlalchemy sql_magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 9.5 MB/s eta 0:00:00


In [2]:
from google.colab import auth
# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [3]:
from google.cloud import secretmanager

def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload of the given secret version and return it.

    Args:
        project_id (str): Google Cloud project ID.
        secret_id (str): ID of the secret to access.
        version_id (str): ID of the version to access.
    Returns:
        str: The secret version's payload, or None if
        the version does not exist.
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode('UTF-8')

openai_key = access_secret_version("sql-autograding", "openai-gpt4-32k", "3")


In [4]:
import openai

openai.api_key = openai_key

In [5]:

from google.colab import files
import io
import pandas as pd

In [6]:
!gsutil cp gs://sql_autograding/cleaned_response.csv cleaned_response.csv

Copying gs://sql_autograding/cleaned_response.csv...
/ [1 files][  2.8 MiB/  2.8 MiB]                                                
Operation completed over 1 objects/2.8 MiB.                                      


In [7]:
data = pd.read_csv('cleaned_response.csv')
data = data.drop(columns='Unnamed: 0')
data.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\n-1 pts for using t...,"SELECT O.OrderID, O.CustomerID\n&#160; &#160; ...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n..."
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN


## Table Schemas

In [8]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [9]:
db_list = ['flights', 'imdb', 'music', 'restaurants', 'facebook', 'northwind', 'collisions']

def db_schema(db_name):
  conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org',
    user = 'student',
    password = 'dwdstudent2015',
    db = db_name,
    encoding = 'utf8mb4')
  engine = create_engine(conn_string)

  with engine.begin() as conn:
    tables = pd.read_sql_query(sql=text('show tables'), con=conn)
    tables = tables.iloc[:,-1].tolist()

    schema_list = []
    for t in tables:
      d = pd.read_sql_query(sql=text(f'describe {t}'), con=conn)
      table_schema = {t:d.loc[:,'Field'].tolist()}
      schema_list.append(table_schema)
  return schema_list

In [10]:
flights_schema = db_schema('flights')
imdb_schema = db_schema('imdb')
music_schema = db_schema('music')
restaurants_schema = db_schema('restaurants')
facebook_schema = db_schema('facebook')
northwind_schema = db_schema('northwind')
collisions_schema = db_schema('collisions')

print(facebook_schema)

[{'Concentration': ['ProfileID', 'Concentration']}, {'FavoriteBooks': ['ProfileID', 'Book']}, {'FavoriteMovies': ['ProfileID', 'Movie']}, {'FavoriteMusic': ['ProfileID', 'Music']}, {'FavoriteTVShows': ['ProfileID', 'TVShow']}, {'Hobbies': ['ProfileID', 'Hobby']}, {'LookingFor': ['ProfileID', 'LookingFor']}, {'Orientation': ['ProfileID', 'InterestedIn']}, {'Profiles': ['ProfileID', 'Name', 'MemberSince', 'LastUpdate', 'School', 'Status', 'Sex', 'Birthday', 'AIM', 'Website', 'PoliticalViews', 'Geography', 'HighSchool', 'HomeTown', 'HomeState', 'Residence', 'CurrentAddress', 'CurrentTown', 'CurrentState']}, {'Relationship': ['ProfileID', 'Status']}, {'chart': ['music', 'gender', 'cnt', 'music_rank']}, {'popular_music': ['music', 'cnt']}, {'popular_music_by_sex': ['music', 'gender', 'cnt']}]


In [11]:
schema_mapping = {
    'flights': flights_schema,
    'imdb': imdb_schema,
    'music': music_schema,
    'restaurants': restaurants_schema,
    'facebook': facebook_schema,
    'northwind': northwind_schema,
    'collisions': collisions_schema,
    "['northwind', 'flights']": [northwind_schema, flights_schema]
}


In [ ]:
# # print(schema_fun(['northwind', 'flights']))
# import re
# def convert_to_list(string):
#     # Remove the square brackets and extra spaces
#     cleaned_string = re.sub(r'\[|\]', '', string).strip()
#     # Split the string into individual elements
#     elements = [elem.strip() for elem in cleaned_string.split(',')]
#     # Return the converted list
#     return elements

In [ ]:
# def schema_fun(db):
#   if isinstance(db, list):
#     # schema_list = []
#     # for name in db:
#     #   schema_list.append(db_schema(name))
#     # return schema_list
#     return [db_schema(name) for name in db]
#   else:
#     return db_schema(db)

In [12]:
data['schema'] = data['Database'].map(schema_mapping)
data.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...,"[[{'Categories': ['CategoryID', 'CategoryName'..."
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\n-1 pts for using t...,"SELECT O.OrderID, O.CustomerID\n&#160; &#160; ...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...,"[[{'Categories': ['CategoryID', 'CategoryName'..."
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","[[{'Categories': ['CategoryID', 'CategoryName'..."
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN,"[[{'Categories': ['CategoryID', 'CategoryName'..."
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN,"[[{'Categories': ['CategoryID', 'CategoryName'..."


In [13]:
data = data.replace('&nbsp;', ' ', regex=True)
data = data.replace('&#160;', ' ', regex=True)

## Group questions

In [14]:
from sklearn.model_selection import train_test_split

In [ ]:
# train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['QuestionId'], random_state=1234)

In [15]:
df = data[data['QuestionId']=='5,119,721']
df.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
158,"Databases for Business Analytics, Spring'23","251,685",Assignment 3: Joins,"295,367",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"select T.track_name, L.album_name, a.artist_na...",1.0,my2204,NaN,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
219,"Databases for Business Analytics, Spring'23","251,685",Assignment 3: Joins,"295,367",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"SELECT track_name, artist_name, album_name\nFR...",0.5,ql2084,NaN,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
289,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 3: Joins,"301,296",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"USE music;\nSHOW tables;\nSELECT track_name, a...",1.0,soj206,NaN,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
332,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 3: Joins,"301,296",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"SELECT T.track_name, A1.artist_name, A2.album_...",0.7,pp1711,\n\n\nAnswer Key\n\n\n\n\n\n\n\nSELECT track_n...,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,The feedback from the TA was \n\n\nAnswer Key\...,"[{'album': ['artist_id', 'album_id', 'album_na..."
420,"Databases for Business Analytics, Spring'23","251,685",Assignment 3: Joins,"295,367",1,"5,119,721","List all the track names, the corresponding al...",Long Answer Question,"Select T.track_name, L.album_name, A.artist_na...",1.0,sek495,NaN,"SELECT track_name, artist_name, album_name\nFR...",True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."


In [16]:
qids = df['QuestionId'].unique()
print(qids)

['5,119,721']


In [ ]:
for id in qids:
  temp = df[df['QuestionId']==id]
  # making test size 1
  train_df, test_df = train_test_split(temp, test_size=1, random_state=1234)
  train_df = train_df.reset_index()
  test_df = test_df.reset_index()

  submissions = ""
  for i,row in train_df.iterrows():
    submission = row['InputUserAnswer']
    grade = row['Score']
    full_grade = row['full_score']
    feedback = row['feedback']
    submissions += f'e{i}. submission: {submission}, grade: {grade}/{full_grade}, feedback: {feedback} \n\n'

  # print(submissions)
  db = train_df.loc[0,'Database']
  tables = train_df.loc[0,'schema']
  question = train_df.loc[0,'Question']
  key = train_df.loc[0,'AnswerKey']

  test = test_df.loc[0,'InputUserAnswer']

  mega_p = f'''
    a. We are working with the {db} database, which has the following tables: {tables}

    b. The request to the student is "{question}"

    c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

    d. The student answer was {test}

    e. Previously, students have submitted these answers and got back these responses and grades: \n{submissions}

    Please provide a grade and feedback for the student
    '''
  print(mega_p)

  # try prompt with and without e part (zero & few shot), training set - 1,2,3...
  # gpt already knows northwind database
  # see what we get back

## Mega Prompt

===============================

a. We are working with the XXX database, which has the following tables:
artist(id, first_name, last_name)
album(id, artist_id, name)
track....

b. The request to the student is "Fetch all the tracks for user X"

c. The model answer is ..... (note that we may have multiple correct
answers, the model answer is just an example)

d. The student answer was .....

e. Previously, students have submitted these answers and got back
these responses and grades:

e1. submission: ..... , grade: ..... , feedback

e2. submission: ..... , grade: ..... , feedback

e3. submission: ..... , grade: ..... , feedback

Please provide a grade and feedback for the student

===============================

In [ ]:
# control the submissions number - zero shot / few shots
def generate_mega_prompt(df, submission_size):
  mega_df = pd.DataFrame(columns=['QuestionId', 'MegaPrompt'])

  qids = df['QuestionId'].unique()

  for id in qids:
    temp = df[df['QuestionId']==id]
    # making test size 1
    train_df, test_df = train_test_split(temp, test_size=1, random_state=1234)
    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    submissions = ""
    for i,row in train_df.iterrows():
      if i >= submission_size:
        break;
      submission = row['InputUserAnswer']
      grade = row['Score']
      full_grade = row['full_score']
      feedback = row['feedback']
      submissions += f'e{i}. submission: {submission}, grade: {grade}/{full_grade}, feedback: {feedback} \n\n'

    # print(submissions)
    db = train_df.loc[0,'Database']
    tables = train_df.loc[0,'schema']
    question = train_df.loc[0,'Question']
    key = train_df.loc[0,'AnswerKey']

    test = test_df.loc[0,'InputUserAnswer']

    mega_p = f'''
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \n{submissions}

      Please provide a grade and feedback for the student
      '''
    new_row = {'QuestionId': id, 'MegaPrompt': mega_p}

    mega_df = mega_df.append(new_row, ignore_index=True)
  return mega_df

In [ ]:
# using all submissions
def generate_mega_prompt_all(df):
  mega_df = pd.DataFrame(columns=['QuestionId', 'MegaPrompt'])

  qids = df['QuestionId'].unique()

  for id in qids:
    temp = df[df['QuestionId']==id]
    # making test size 1
    # drop the question if it has only 1 record now
    if len(temp)<=1:
      continue;
    train_df, test_df = train_test_split(temp, test_size=1, random_state=1234)
    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    submissions = ""
    for i,row in train_df.iterrows():
      submission = row['InputUserAnswer']
      grade = row['Score']
      full_grade = row['full_score']
      feedback = row['feedback']
      submissions += f'e{i}. submission: {submission}, grade: {grade}/{full_grade}, feedback: {feedback} \n\n'

    # print(submissions)
    db = train_df.loc[0,'Database']
    tables = train_df.loc[0,'schema']
    question = train_df.loc[0,'Question']
    key = train_df.loc[0,'AnswerKey']

    test = test_df.loc[0,'InputUserAnswer']

    mega_p = f'''
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \n{submissions}

      Please provide a grade and feedback for the student
      '''
    new_row = {'QuestionId': id, 'MegaPrompt': mega_p}

    mega_df = mega_df.append(new_row, ignore_index=True)
  return mega_df

In [ ]:
# for all questions each with all submissions => new df
mega_df_all = generate_mega_prompt_all(data)
mega_df_all.head()

In [ ]:
print(mega_df_all.loc[0, 'MegaPrompt'])

In [ ]:
# select a specific question and control submission size
df = data[data['QuestionId']=='5,168,443']

df0 = generate_mega_prompt(df, 0)
df0.head()

<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)


,QuestionId,MegaPrompt
0,"5,168,443",\n a. We are working with the flights dat...


In [ ]:
df5 = generate_mega_prompt(df, 5)
df10 = generate_mega_prompt(df, 10)
df20 = generate_mega_prompt(df, 20)
df_all = generate_mega_prompt(df, len(df))

<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)
<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)
<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)
<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)


## GPT4

In [ ]:
def GPT4_generation(prompt):
  response = openai.ChatCompletion.create(
    model="gpt-4-32k",
    messages=[{"role": "user", "content": prompt}],
    n = 1,
    stream = False,
    temperature=0.0,
    max_tokens=600,
    top_p = 1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
    # stop = ["Q:"]
  )
  return response['choices'][0]['message']['content']

In [ ]:
print(GPT4_generation(df0.loc[0,'MegaPrompt']))

Grade: 9/10

Feedback: Your answer is almost perfect. You have correctly used the RANK() function to calculate the national and state rank. However, you have used 'total_airport_passengers' instead of 'total_passengers' as in the model answer. Please make sure to use the correct column names. Otherwise, your SQL syntax and join operation are correct. Good job!


In [ ]:
print(GPT4_generation(df5.loc[0,'MegaPrompt']))

The student's answer is correct. They have correctly used the RANK() function to calculate the national and state ranks based on the total number of passengers. They have also correctly joined the m_airports table to get the state of each airport. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function to calculate the national and state ranks. Your join operation is also correct. Keep up the good work!


In [ ]:
print(GPT4_generation(df10.loc[0,'MegaPrompt']))

Grade: 4.0/4.0

Feedback: Your answer is correct. You have successfully used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. Your join operation is also correct, linking the 'origin' field in the 'passengers_per_airport' table with the 'airport' field in the 'm_airports' table. Good job!


In [ ]:
print(GPT4_generation(df20.loc[0,'MegaPrompt']))

The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the m_airports table to get the state of each airport. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function and joined the necessary tables to get the desired output. Keep up the good work!


In [ ]:
print(GPT4_generation(df_all.loc[0,'MegaPrompt']))

The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the passengers_per_airport and m_airports tables on the 'origin' and 'airport' columns respectively. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. Your join operation is also correct. Keep up the good work!


In [ ]:
def test_shots(qid, n):
  df = data[data['QuestionId']==qid]
  df_n = generate_mega_prompt(df, n)
  # df_n.head()
  print(f'{n} submissions:')
  print(GPT4_generation(df_n.loc[0,'MegaPrompt']))


In [ ]:
for i in range(0, 21, 5):
  test_shots('5,168,443', i)

<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)


0 submissions:
The student's answer is correct. The student correctly used the RANK() function to calculate the national and state ranks. The student also correctly joined the passengers_per_airport and m_airports tables on the airport column. The only difference between the student's answer and the model answer is the naming of the columns, but this does not affect the correctness of the answer. 

Grade: 10/10

Feedback: Great job! You correctly used the RANK() function and joined the tables correctly. Keep up the good work!
5 submissions:


<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)


The student's answer is correct. They have correctly used the RANK() function to calculate the national and state ranks based on the total number of passengers. They have also correctly joined the m_airports table to get the state of each airport. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function to calculate the national and state ranks. Your join operation is also correct. Keep up the good work!
10 submissions:


<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)


The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the passengers_per_airport and m_airports tables on the airport column. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function and joined the tables correctly. Keep up the good work!
15 submissions:


<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)


Grade: 4.0/4.0

Feedback: Your SQL query is correct. You have successfully calculated the national and state rank for each airport based on the number of departing passengers. Good job on using the RANK() function and joining the tables correctly. Keep up the good work!
20 submissions:


<ipython-input-18-d78b26898815>:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mega_df = mega_df.append(new_row, ignore_index=True)


The student's answer is correct. They have correctly used the RANK() function to calculate the national and state rank of each airport based on the number of departing passengers. They have also correctly joined the m_airports table to get the state of each airport. 

Grade: 4.0/4.0

Feedback: Great job! You have correctly used the RANK() function and joined the necessary tables to get the desired output. Keep up the good work!


In [ ]:
mega_df_all.shape
mega_df_all.head(20)

,QuestionId,MegaPrompt
0,"4,969,729",\n a. We are working with the ['northwind...
1,"4,969,713",\n a. We are working with the ['northwind...
2,"4,969,711",\n a. We are working with the ['northwind...
3,"4,969,710",\n a. We are working with the ['northwind...
4,"4,969,709",\n a. We are working with the ['northwind...
5,"4,969,708",\n a. We are working with the ['northwind...
6,"4,969,707",\n a. We are working with the ['northwind...
7,"4,969,706",\n a. We are working with the ['northwind...
8,"4,969,705",\n a. We are working with the ['northwind...
9,"5,119,646",\n a. We are working with the music datab...


In [ ]:
mega_df_all.shape

(56, 2)

In [ ]:
from tqdm import tqdm

In [ ]:
for i, p in tqdm(mega_df_all.iterrows()):
  # print(i, len(p['MegaPrompt'].split()))
  p['feedback'] = GPT4_generation(p['MegaPrompt'])

mega_df_all.head()

56it [04:14,  4.55s/it]


,QuestionId,MegaPrompt
0,"4,969,729",\n a. We are working with the ['northwind...
1,"4,969,713",\n a. We are working with the ['northwind...
2,"4,969,711",\n a. We are working with the ['northwind...
3,"4,969,710",\n a. We are working with the ['northwind...
4,"4,969,709",\n a. We are working with the ['northwind...


In [ ]:
mega_df_all['feedback'] = mega_df_all['MegaPrompt'].apply(GPT4_generation)
mega_df_all.head()